In [87]:
import numpy as np
import pandas as pd
import requests
import random

# FVtyMiv9TXSjy0x4O9xmVYE2iksnAlYeEXPaD2sm

In [88]:
eia_api_key = 'FVtyMiv9TXSjy0x4O9xmVYE2iksnAlYeEXPaD2sm'

In [89]:
# api_url = f'https://api.eia.gov/v2/electricity/rto/region-data/data/?frequency=hourly&data[0]=value&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={eia_api_key}'

# headers = {
#     "X-Params": '{"frequency": "hourly", "data": ["value"], "facets": {}, "start": "2022-10-01T00", "end": "2023-10-01T00", "sort": [{"column": "period", "direction": "desc"}], "offset": 0, "length": 5000}'
# }

# response = requests.get(api_url, headers=headers)

# if response.status_code == 200:
#     data = response.json()

# else:
#     print(f"Error: {response.status_code} - {response.text}")

In [90]:
# locations = []
# types = [item['type-name'] for item in data["response"]["data"] if item['type-name'] == 'Demand']

# Demand = [item["value"] for item in data["response"]["data"] if item['type-name'] == 'Demand']

# Total_interchange = [item["value"] for item in data["response"]["data"] if item['type-name'] == 'Total Interchange']

# Net_generation = [item["value"] for item in data["response"]["data"] if item['type-name'] == 'Net Generation']

# Day_ahead = [item["value"] for item in data["response"]["data"] if item['type-name'] == 'Day-ahead Demand Forecast']



# all_values = [item["type-name"] for item in data["response"]["data"]]


# desired_responses = ["Day-ahead demand forecast", "Net generation", "Demand", "Total interchange"]

# # Create separate lists for each response
# response_data = {}
# for response in desired_responses:
#     response_data[response] = [item["value"] for item in data["response"]["data"] if item['type-name'] == response]

# # Print the data for each response
# for response, values in response_data.items():
#     print(response)
#     for value in values:
#         print(value)
#     print('\n')

# # for value in all_values:
# #   print(value)

# # for i, (type, value) in enumerate(zip(types, Demand)):
# #   print(i, type, value)
# #   print('\n')
# #print(types, values)

In [91]:
max_len = max(len(types), len(Demand), len(Total_interchange), len(Net_generation), len(Day_ahead))

# Pad lists with NaN to make them of the same length
def pad_list(lst, length):
    if len(lst) < length:
        return lst + [np.nan] * (length - len(lst))
    return lst

types = pad_list(types, max_len)
Demand = pad_list(Demand, max_len)
Total_interchange = pad_list(Total_interchange, max_len)
Net_generation = pad_list(Net_generation, max_len)
Day_ahead = pad_list(Day_ahead, max_len)

# Create the DataFrame
data = {
    'Type': types,
    'Demand': Demand,
    'Total Interchange': Total_interchange,
    'Net Generation': Net_generation,
    'Day-ahead Demand Forecast': Day_ahead
}

df = pd.DataFrame(data)

# Print the DataFrame
print(df)

        Type  Demand  Total Interchange  Net Generation  \
0     Demand     301                NaN             NaN   
1     Demand    1255                NaN             NaN   
2     Demand    5553                NaN             NaN   
3     Demand   12774                NaN             NaN   
4     Demand     169                NaN             NaN   
...      ...     ...                ...             ...   
1201  Demand    4300                NaN             NaN   
1202  Demand     204                NaN             NaN   
1203  Demand   28913                NaN             NaN   
1204  Demand   30636                NaN             NaN   
1205  Demand    3780                NaN             NaN   

      Day-ahead Demand Forecast  
0                           NaN  
1                           NaN  
2                           NaN  
3                           NaN  
4                           NaN  
...                         ...  
1201                        NaN  
1202               

In [92]:
import pandas as pd
import numpy as np

In [93]:
df = pd.read_csv("EIA930_BALANCE_2023_Jul_Dec(3).csv")
df = df.loc[:, 'Data Date':'Total Interchange (MW)']

/var/folders/7d/hyzwnh6n6_533550djz9zsbc0000gn/T/ipykernel_62730/2066472169.py:1: DtypeWarning: Columns (15,16,17,18,19,21,22,23,25,26,27,29,30,31,32,33,34,35,36,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("EIA930_BALANCE_2023_Jul_Dec(3).csv")


In [94]:
def generate_fake_cost():
    # Generate random values around $50 with some variation
    base_cost = 50
    variation = random.uniform(-20, 20) # Varaition, can adjust
    return base_cost + variation

# Create the 'Electricity Cost ($)' column with fake data
df['Electricity Cost ($)'] = [generate_fake_cost() for _ in range(len(df))]

In [95]:
def calculate_charging_rate(electricity_cost, median_cost):
    #Calculate deviation from the median
    deviation = electricity_cost - median_cost
    
    #Charge faster when cost is below the median, slower when above
    if deviation < 0:
        return 1.0  # Charge at full rate
    else:
        return max(0.2, 0.2 + (1.0 - 0.2) * (1 - deviation / median_cost))  # Linearly decrease charging rate as cost deviates

# Calculate the median electricity cost
median_cost = df['Electricity Cost ($)'].median()

df['Charging Rate'] = df['Electricity Cost ($)'].apply(lambda x: calculate_charging_rate(x, median_cost))

In [99]:
df.head()

,Data Date,Hour Number,Local Time at End of Hour,UTC Time at End of Hour,Demand Forecast (MW),Demand (MW),Net Generation (MW),Total Interchange (MW),Electricity Cost ($),Charging Rate
0,07/01/2023,1,07/01/2023 1:00:00 AM,07/01/2023 6:00:00 AM,"3,074","3,102","2,314",-788,51.492389,0.974035
1,07/01/2023,2,07/01/2023 2:00:00 AM,07/01/2023 7:00:00 AM,"2,866","2,902","2,239",-663,61.438088,0.814500
2,07/01/2023,3,07/01/2023 3:00:00 AM,07/01/2023 8:00:00 AM,"2,702","2,771","2,286",-485,55.970809,0.902199
3,07/01/2023,4,07/01/2023 4:00:00 AM,07/01/2023 9:00:00 AM,"2,597","2,654","2,347",-307,34.080648,1.000000
4,07/01/2023,5,07/01/2023 5:00:00 AM,07/01/2023 10:00:00 AM,"2,506","2,542","2,281",-261,66.453260,0.734054


In [97]:
print(df["Charging Rate"].min())

0.6771662647856045
